In [6]:
from config_cvt.default import get_config
from cvt import generate_model
config=get_config('./config_cvt/config.yaml')
paddle_model = generate_model(config)
paddle_model.eval()


merging config from ./config_cvt/config.yaml


In [7]:
import torchvision.models as models
import torch

torch_model= models.squeezenet1_1(pretrained=False)
torch_model.load_state_dict= torch.load('CvT-13-224x224-IN-1k.pth',map_location=torch.device('cpu'))


In [8]:
import numpy as np
import paddle
x = np.random.randn(2, 3, 224, 224).astype('float32')
x_paddle = paddle.to_tensor(x)
x_torch = torch.Tensor(x).to('cpu')

In [9]:
out_torch = torch_model(x_torch)

out_torch


tensor([[0.2856, 0.2334, 0.0756,  ..., 0.3653, 0.0054, 0.5043],
        [0.2404, 0.2001, 0.0951,  ..., 0.3266, 0.0075, 0.5672]],
       grad_fn=<ReshapeAliasBackward0>)

In [10]:
out_paddle = paddle_model(x_paddle)

Tensor(shape=[2, 64, 56, 56], dtype=float32, place=CPUPlace, stop_gradient=True,
       [[[[ 0.26045838, -0.43781328,  1.21074760, ..., -1.55346048,
            1.04184794,  1.86613953],
          [-0.13499232,  0.27428111, -2.36626005, ...,  0.42008662,
            2.19770265,  0.30858502],
          [-0.29843548,  0.26569623, -0.17962067, ..., -0.87878090,
           -0.11445840, -0.28694659],
          ...,
          [-0.08478997, -0.27485076,  1.14770591, ...,  0.42022875,
            0.01334239,  1.72689688],
          [-0.81171614, -0.83647680,  0.09441633, ..., -0.46618333,
           -0.62700701,  0.83925331],
          [-0.12934899, -0.92572141,  1.25967491, ...,  0.01080432,
           -1.45990431,  0.08035537]],

         [[-1.33436513,  0.32139024, -2.25323439, ...,  1.34216940,
           -0.68781179, -0.30354151],
          [ 0.03609634, -0.75701541,  0.03071276, ...,  0.25675142,
           -0.84926271, -0.22070612],
          [ 0.41238073, -1.16012013, -0.80653757, ...,

NotImplementedError: 